# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from nlppen.extraccion.utils.extraerFechaRecibido import ExtraerFecha
from pyspark.sql.functions import length

import pandas as pd


## Configuracion Spark

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "6g")
         .config("spark.executor.cores", "1")
         .config("spark.driver.memory", "12g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/11/19 22:32:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:
ejecutar = False
if ejecutar:
    terminos = {
        'seguimiento': [r'\bseguimiento\b'],
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructuralesNormal')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
    print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
    estructurales = SentenciasEstructurales(seleccion)

# Procesamiento de todas las variables de sentencias estructurales

## La siguiente celda ejecuta el cálculo de todas las variables definidas para las sentencias estructurales

Nota: Modificar la bandera "ejecutar" para aplicar todo el procesamiento.

In [56]:
from pyspark.sql.types import *

if ejecutar:
    columnas = {
        'se ordena PER' : ArrayType(StringType()),
        'se ordena LOC' : ArrayType(StringType()),
        'se ordena ORG' : ArrayType(StringType()),
        'se ordena MISC' : ArrayType(StringType()),
        'se ordena GPE' : ArrayType(StringType()),
        'se ordena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeOrdena(columnas, True, True)

    columnas = {
        'se condena PER' : ArrayType(StringType()),
        'se condena LOC' : ArrayType(StringType()),
        'se condena ORG' : ArrayType(StringType()),
        'se condena MISC' : ArrayType(StringType()),
        'se condena GPE' : ArrayType(StringType()),
        'se condena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeCondena(columnas, True, True)

    columnas = {
        'extension sentencia' : IntegerType(),
        'extension por lo tanto' : IntegerType()
    }
    estructurales.extraerExtension(columnas, True)

    columnas = {
        'plazosDefinidos' : ArrayType(TimestampType())
    }
    estructurales.plazosDefinidos(columnas, True)

    columnas = {
        'FechaSolicitud' : TimestampType(),
    }

    estructurales.extrarFechaRecibido(columnas, True)

    columnas = {
        'num resolucion' : StringType()
    }

    estructurales.extraerNumeroSentencia(columnas, True)

    columnas = {
        'inst internacionales' : ArrayType(StringType())
    }

    estructurales.extraerInstrumentosInternacionales(columnas, True)

    columnas = {
        'derechos Norm' : ArrayType(StringType()),
        'derechos GenXPat' : ArrayType(StringType())
    }
    estructurales.extraerDerechos(columnas, True)

    columnas = {
        'derechos Acotados' : ArrayType(StringType()),
        'derechos General' : ArrayType(StringType()),
        'derechos Fundamental' : ArrayType(StringType()),
        'derechos Humano' : ArrayType(StringType())
    }

    estructurales.extraerDerechosSinNormalizar(columnas, True)
    
    columnas = {
        'citasFechas' : MapType(StringType(), TimestampType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, True)
    
    sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
    columnas = {
        'id_sentencia' : StringType(),
        'num_doc_oficial' : StringType(),
        'expediente_oficial' : StringType()
    }
    estructurales.agregarIDSentencia(columnas, sentencias, True)
    #Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
    estructurales.seleccion.guardarDatos()

## Guardar los resultados del preprocesamiento por año

In [57]:
if ejecutar:
    df = spark.read.parquet(seleccion.datasets_path+"/terminos.parquet")
    writer = df.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')


## Cargar los resultados preprocesados y divididos por año

In [72]:
if not ejecutar:
    seleccion = Seleccion({}, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesNormal')
    estructurales = SentenciasEstructurales(seleccion)

In [73]:
estructurales.seleccion.sdf.toPandas().head(5)

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,Fechahora_ents,Redactor_ents,CitaSentencias_ents,Reglamento_ents,termino_lst,...,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion,inst_internacionales,derechos_Norm,derechos_GenXPat,derechos_Acotados,derechos_General,derechos_Fundamental,derechos_Humano,id_sentencia,num_doc_oficial,expediente_oficial,anno
0,2018112102451287_19152.html,*180169790007CO* Exp: 18-016979-0007-CO Res. Nº 2018019152 SALA CONSTITUCION...,SALUD,0.994687,180169790007CO,2018-11-16 09:15:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas quince minutos del dieciseis de noviembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],...,2641,[1970-04-01 00:00:00],NaT,2018019152,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho fundamental a la salud, derecho a la vida, dere...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho a ...","[derecho en una parte de la población, derecho fundamental a la salud, derec...",[derecho fundamental a la salud],[derecho humano que tiene el derecho a la salud de todos los ciudadanos],sen-1-0007-893680,19152,180169790007CO,2018
1,2018112063619_17421.html,*180157550007CO* Exp: 18-015755-0007-CO Res. Nº 2018017421 SALA CONSTITUCION...,SALUD,1.000000,180157550007CO,2018-10-19 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar,False,[las nueve horas treinta minutos del diecinueve de octubre de dos mil diec...,[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],...,2252,None,2018-10-07 00:00:00,2018017421,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho a la salud, derecho a la salud, derecho a la sa...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho hu...","[derecho a la igualdad de trato ante, derecho en una parte de la población, ...",None,[derecho humano que tiene el derecho a],sen-1-0007-860153,17421,180157550007CO,2018
2,201812701524278_20041.html,*180178880007CO* Exp: 18-017888-0007-CO Res. Nº 2018020041 SALA CONSTITUCION...,TRABAJO,0.632804,180178880007CO,2018-11-30 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,False,[las nueve horas treinta minutos del treinta de noviembre de dos mil diecioc...,[Fernández Argüello],"[2018-2038, 2016-003772, 2017-018586]",[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Con lugar parcial, Sin lugar]",...,2130,None,2018-11-09 16:05:00,2018020041,None,"[Derecho Fundamental al Salario, Derecho Fundamental al Salario, Derecho Fun...","[derecho constitucional al salario, derecho constitucional al salario, derec...","[derecho fundamental, derecho constitucional]","[derecho constitucional al salario, derecho fundamental al salario]",[derecho fundamental al salario],None,sen-1-0007-894916,20041,180178880007CO,2018
3,201911004458949_21429.html,*180188180007CO* Exp: 18-018818-0007-CO Res. Nº 2018021429 SALA CONSTITUCION...,TRABAJO,0.390122,180188180007CO,2018-12-21 09:20:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas veinte minutos del veintiuno de diciembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Con lugar, Sin lugar]",...,1978,None,2018-11-26 11:28:00,2018021429,None,None,None,None,None,None,None,sen-1-0007-896941,21429,180188180007CO,2018
4,20189180242172_14652.html,*180063820007CO* Exp: 18-006382-0007-CO Res. Nº 2018014652 SALA CONSTITUCION...,PETICION,0.086251,180063820007CO,2018-09-07 09:20:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,True,[las nueve horas veinte minutos del siete de setiembre de dos mil dieciocho .],[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Co

## Ejecutar un criterio

In [68]:
if not ejecutar:
    sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
    columnas = {
        'id_sentencia' : StringType(),
        'num_doc_oficial' : StringType(),
        'expediente_oficial' : StringType()
    }
    estructurales.agregarIDSentencia(columnas, sentencias, True)
    s = estructurales.seleccion.sdf

In [69]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)
estructurales.seleccion.sdf.toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,Fechahora_ents,Redactor_ents,CitaSentencias_ents,Reglamento_ents,termino_lst,...,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion,inst_internacionales,derechos_Norm,derechos_GenXPat,derechos_Acotados,derechos_General,derechos_Fundamental,derechos_Humano,anno,id_sentencia,num_doc_oficial,expediente_oficial
0,2018112102451287_19152.html,*180169790007CO* Exp: 18-016979-0007-CO Res. Nº 2018019152 SALA CONSTITUCION...,SALUD,0.994687,180169790007CO,2018-11-16 09:15:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas quince minutos del dieciseis de noviembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],...,2641,[1970-04-01 00:00:00],NaT,2018019152,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho fundamental a la salud, derecho a la vida, dere...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho a ...","[derecho en una parte de la población, derecho fundamental a la salud, derec...",[derecho fundamental a la salud],[derecho humano que tiene el derecho a la salud de todos los ciudadanos],2018,sen-1-0007-893680,19152,180169790007CO
1,2018112063619_17421.html,*180157550007CO* Exp: 18-015755-0007-CO Res. Nº 2018017421 SALA CONSTITUCION...,SALUD,1.000000,180157550007CO,2018-10-19 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar,False,[las nueve horas treinta minutos del diecinueve de octubre de dos mil diec...,[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],...,2252,None,2018-10-07 00:00:00,2018017421,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho a la salud, derecho a la salud, derecho a la sa...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho hu...","[derecho a la igualdad de trato ante, derecho en una parte de la población, ...",None,[derecho humano que tiene el derecho a],2018,sen-1-0007-860153,17421,180157550007CO
2,201812701524278_20041.html,*180178880007CO* Exp: 18-017888-0007-CO Res. Nº 2018020041 SALA CONSTITUCION...,TRABAJO,0.632804,180178880007CO,2018-11-30 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,False,[las nueve horas treinta minutos del treinta de noviembre de dos mil diecioc...,[Fernández Argüello],"[2018-2038, 2016-003772, 2017-018586]",[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Con lugar parcial, Sin lugar]",...,2130,None,2018-11-09 16:05:00,2018020041,None,"[Derecho Fundamental al Salario, Derecho Fundamental al Salario, Derecho Fun...","[derecho constitucional al salario, derecho constitucional al salario, derec...","[derecho fundamental, derecho constitucional]","[derecho constitucional al salario, derecho fundamental al salario]",[derecho fundamental al salario],None,2018,sen-1-0007-894916,20041,180178880007CO
3,201911004458949_21429.html,*180188180007CO* Exp: 18-018818-0007-CO Res. Nº 2018021429 SALA CONSTITUCION...,TRABAJO,0.390122,180188180007CO,2018-12-21 09:20:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas veinte minutos del veintiuno de diciembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Con lugar, Sin lugar]",...,1978,None,2018-11-26 11:28:00,2018021429,None,None,None,None,None,None,None,2018,sen-1-0007-896941,21429,180188180007CO
4,20189180242172_14652.html,*180063820007CO* Exp: 18-006382-0007-CO Res. Nº 2018014652 SALA CONSTITUCION...,PETICION,0.086251,180063820007CO,2018-09-07 09:20:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,True,[las nueve horas veinte minutos del siete de setiembre de dos mil dieciocho .],[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Co

In [70]:
s = estructurales.seleccion.sdf
nueva = s.filter(s.num_doc_oficial.isNotNull()).count()
antigua = s.filter(s.num_resolucion.isNotNull()).count()
print("Acc", nueva/antigua)

Acc 0.9948415622697127


In [38]:

pd.set_option('display.max_rows', 20)
newData = s.filter(s.num_resolucion.isNotNull() & s.num_doc_oficial.isNull())
print("Faltantes", newData.count())
newData.toPandas()[['expediente', 'id_sentencia', 'num_resolucion', 'num_doc_oficial']].head(50)


Faltantes 7


/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,expediente,id_sentencia,num_resolucion,num_doc_oficial
0,080030410007CO,None,2008-003700,None
1,070080760007CO,None,20070-09825,None
2,1922V96,None,1922-V-96,None
3,6404P95,None,2330-96,None
4,2772C93,None,2772-C-93,None
5,2423C92,None,2423-C-92,None
6,3877C92,None,3877-C-92,None


In [71]:
#writer = estructurales.seleccion.sdf.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')

In [ ]:
"""from pyspark.sql.functions import explode,posexplode, desc
print("Sentencias con citas (antiguas)", s.filter(s.CitaSentencias_ents.isNotNull()).count())
print("Sentencias con citas (nuevas)", s.filter(s.citasFechas.isNotNull()).count())

print("Citas que no fueron encontradas previamente", s.filter(s.CitaSentencias_ents.isNull() & s.citasFechas.isNotNull()).count())
print("Citas que no fueron encontradas ahora en comparación con las antiguas", s.filter(s.CitaSentencias_ents.isNotNull() & s.citasFechas.isNull()).count())

totalAntiguas = (s.filter(s.CitaSentencias_ents.isNotNull())
    .select(explode('CitaSentencias_ents').alias('CitaSentencias_ents')))
totalNuevas = (s.filter(s.citasFechas.isNotNull())
    .select(explode('citasFechas')))

totalConFecha = totalNuevas.filter(totalNuevas.value.isNotNull())
print("Total de citas (antiguas)", totalAntiguas.count())
print("Total de citas (nuevas)", totalNuevas.count())
print("Total de citas con fecha (nuevas)", totalConFecha.count())
print("Porcentaje de extracción de fechas ", totalConFecha.count()/totalNuevas.count())
"""


# Cargar procesamiento de sentencias estructurales.

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)

# Zona de trabajo (ignorar)

In [41]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)


In [ ]:
import re
s = estructurales.seleccion.sdf


In [ ]:
sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
sentencias.head(5)

In [ ]:
txt = "002786-2009"
annoInicio = "-"
exp = re.compile(annoInicio,  re.X | re.M | re.I)
for m in exp.finditer(txt):
    print(m.group(0))
print(exp.split(txt))

In [ ]:
num = int('0002546')
print(num)

In [49]:
#Expediente 2772C93
#['2772', 'C', '93']


s = sentencias[sentencias.numeroDocumento == 2772].sort_values(by="anno")
s
#s[s.anno == 1996]

,Unnamed: 0,anno,expediente,id,numeroDocumento,sentenciasRelacionadas
61468,61468,1992,920023870007CO,sen-1-0007-121338,2772,NaN
4838,4838,1993,930022020007CO,sen-1-0007-122019,2772,NaN
246235,246235,1994,920017130007CO,sen-1-0007-95913,2772,NaN
102681,102681,1995,950024780007CO,sen-1-0007-101234,2772,NaN
201569,201569,1996,960022760007CO,sen-1-0007-105220,2772,NaN
120165,120165,1997,970023200007CO,sen-1-0007-125534,2772,NaN
118995,118995,1998,980027330007CO,sen-1-0007-114929,2772,NaN
137098,137098,1999,990021800007CO,sen-1-0007-200519,2772,NaN
39573,39573,2000,000018330007CO,sen-1-0007-136210,2772,NaN
307038,307038,2001,010029780007CO,sen-1-0007-157874,2772,NaN


In [45]:
def comprobarNumResolucion(numRes, sentenciasCSV, expediente):
    idSentencia = None
    numDoc      = None
    print("Expediente", expediente)
    if numRes is not None:
        
        year = None
        numResolucionSplitted = None
        if numRes.isdigit():
            #Es numero unicamente, debo buscar donde está el año
            regExp = "((?:19[0-9][0-9])|(?:20[0-9][0-9]))"
            exp = re.compile(regExp,  re.X | re.M | re.I)
            splitNumRes = exp.split(numRes, 1)
            if(len(splitNumRes) > 2):
                if exp.match(splitNumRes[1]):
                    year = splitNumRes[1]
                    numResolucionSplitted = splitNumRes[2]
                else:
                    numResolucionSplitted = splitNumRes[1]
                    year = splitNumRes[2]
                
        else:
            regExp = "-"
            exp = re.compile(regExp,  re.X | re.M | re.I)
            splitNumRes = exp.split(numRes)
            print(splitNumRes)

            #Año a la izquierda 97
            regExp = "[0-9][0-9]$"
            exp = re.compile(regExp,  re.X | re.M | re.I)
            if exp.match(splitNumRes[0]):
                year = splitNumRes[0]
                if year[0] == '0' or year[0] == '1' or year[0] == '2':
                    year = str(2000+int(year))
                else:
                    year = str(1900+int(year))
                
                numResolucionSplitted = splitNumRes[1]
            else:
                #Año a la derecha 97
                regExp = "[0-9][0-9]$"
                exp = re.compile(regExp,  re.X | re.M | re.I)
                if exp.match(splitNumRes[1]):
                    year = splitNumRes[1]
                    if year[0] == '0' or year[0] == '1' or year[0] == '2':
                        year = str(2000+int(year))
                    else:
                        year = str(1900+int(year))
                    
                    numResolucionSplitted = splitNumRes[0]
                else:
                    regExp = "((?:19[0-9][0-9])|(?:20[0-9][0-9]))"
                    exp = re.compile(regExp,  re.X | re.M | re.I)
                    if(len(splitNumRes) > 1):
                        if exp.match(splitNumRes[0]):
                            #Año a la izquierda 1998
                            year = splitNumRes[0]
                            numResolucionSplitted = splitNumRes[1]
                        else:
                            #Año a la derecha 1998
                            year = splitNumRes[1]
                            numResolucionSplitted = splitNumRes[0]               
        try:
            numResolucionSplitted = int(numResolucionSplitted)
            year = int(year)
        except:
            pass
        
        print(numResolucionSplitted)
        print(year)
        #print("Num resolucion", numRes)
        #print("Resolucion", numResolucionSplitted, "Año", year) 
        #print("Anno", year)
        #print("Número de resolucion spliteado", numResolucionSplitted)
        sentenciaFiltered = sentenciasCSV[sentenciasCSV.numeroDocumento == numResolucionSplitted]
        sentenciaFiltered = sentenciaFiltered[sentenciaFiltered.anno == year]
        sentenciaFiltered = sentenciaFiltered[sentenciaFiltered.expediente == expediente]
        lenFilter = len(sentenciaFiltered)
        if lenFilter == 1:
            idSentencia = str(sentenciaFiltered.iloc[0, 3])
            numDoc = str(sentenciaFiltered.iloc[0, 4])
            #print(idSentencia, numDoc)
            #print("Find it")

In [46]:
import re

In [47]:
citas = newData.toPandas()
#citas = s.filter(s.CitaSentencias_ents.isNotNull()).toPandas()
nlp = spark_get_spacy("es_core_news_lg")
#size = len(citas)
size = 1
for i in range(0, 10):
    sent = citas.iloc[i, 4]
    txt = citas.iloc[i, 1]
    num_resolucion = citas.iloc[i, -13]
    citasExtraidas  = citas.iloc[i, -2]
    print("******************************")
    print(sent)
    
    print(num_resolucion)
    #print("Citas")
    #print("I: ", i)
    #print(txt)
    #if sent == "170068340007CO":
    #    print(txt)
    #print(citas.iloc[i, 12])
    #print(citasExtraidas)
    comprobarNumResolucion(num_resolucion, sentencias, sent)

******************************
080030410007CO
2008-003700
Expediente 080030410007CO
['2008', '003700']
3700
2008
******************************
070080760007CO
20070-09825
Expediente 070080760007CO
['20070', '09825']
9825
20070
******************************
1922V96
1922-V-96
Expediente 1922V96
['1922', 'V', '96']
V
1922
******************************
6404P95
2330-96
Expediente 6404P95
['2330', '96']
2330
1996
******************************
2772C93
2772-C-93
Expediente 2772C93
['2772', 'C', '93']
2772
C
******************************
2423C92
2423-C-92
Expediente 2423C92
['2423', 'C', '92']
2423
C
******************************
3877C92
3877-C-92
Expediente 3877C92
['3877', 'C', '92']
3877
C


/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


IndexError: single positional indexer is out-of-bounds